# **FBA.gov 조달서비스 크롤링**
- https://www.fbo.gov/index.php?s=opportunity&amp=&pageID=3
- https://www.fbo.gov/index.php?s=opportunity&mode=list&tab=list&tabmode=list&pp=100
- https://www.fbo.gov/index.php?s=opportunity&mode=list&tab=list&tabmode=list&pp=100&pageID=2
- https://www.fbo.gov/index.php?s=opportunity&mode=list&tab=list&tabmode=list

## **1 전체 페이지에서 개별 목록 수집하기**
https://github.com/SmugZombie/FBO_Parser/blob/master/fbo_parse.py

```html
<div class="solt">Janitorial Service </div>,   # 입찰서 제목
<div class="soln">70FBR919Q00000072 </div>,    # 입찰서 번호
<div class="solcc">S -- Utilities and housekeeping services </div> # FSC CODE
```

In [1]:
import requests
def html_download(url, method="get", params=None, data=None):
    userAgent = {"user-agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
    return requests.request(method, url, params=params, data=data, headers= userAgent)

from bs4 import BeautifulSoup
url  = "https://www.fbo.gov/index.php?s=opportunity&mode=list&tab=list&tabmode=list&pp=100"
html = html_download(url)
dom  = BeautifulSoup(html.text, "html.parser") # 오류시 
type(html.text), type(dom)

(str, bs4.BeautifulSoup)

## **2 Row 기준으로 추출하기**
https://github.com/SmugZombie/FBO_Parser/blob/master/fbo_parse.py

```html
<tr id="row_0" class="lst-rw lst-rw-first lst-rw-odd">
<tr id="row_1" class="lst-rw lst-rw-first lst-rw-odd">   
```

In [2]:
# 정규식을 사용한 Token 을 활용하면 다양한 가변조합을 선택 가능
import re
token_id   = re.compile("row_\d{1,2}")
link_pages = [_ for _ in dom.find_all("tr", {"id":token_id})]
len(link_pages), len(link_pages[0].find_all('td'))

(100, 4)

In [27]:
result = []
for link in link_pages:
    row_result = []
    tags = link.find_all('td')
    row_result += [tags[0].find("", {"class":_}).text.strip() 
                   for _ in ['soln','solt','solcc']] +\
                [requests.compat.urljoin(url, tags[0].a['href'])] +\
                [tags[1].text.strip()] + [tags[2].text.strip()] + [tags[3].text.strip()]
    result.append(row_result)

In [28]:
import pandas as pd
df = pd.DataFrame(result)
df.columns = ['입찰번호','입찰제목', 'FCS CODE','detail_page', 'Agency/Office/Location', 'Type Sort/Set-aside Sort', 'Posted On Sort']
df.head()

,입찰번호,입찰제목,FCS CODE,detail_page,Agency/Office/Location,Type Sort/Set-aside Sort,Posted On Sort
0,,J&A Aerosmith Maintenance Calibration,66 -- Instruments & laboratory equipment,https://www.fbo.gov/index.php?s=opportunity&mo...,Department of the Army Army Contracting Comman...,Justification and Approval (J&A) / Cancelled,"Sep 04, 2019"
1,W9098S19T0104,Air Inlet Duct Work,"41 -- Refrigeration, air-conditioning & air ci...",https://www.fbo.gov/index.php?s=opportunity&mo...,Department of the Army Army Contracting Comman...,Solicitation / Woman Owned Small Business,"Sep 04, 2019"
2,2840-00-157-3633RV,"TF33 Engine, P100 1st Stage Fan Blade Sets, NS...","28 -- Engines, turbines & components",https://www.fbo.gov/index.php?s=opportunity&mo...,Department of the Air Force Air Force Materiel...,Sources Sought,"Sep 04, 2019"
3,409527R1_2019,323111SIS Stateside Superintendent Kits.,"T -- Photographic, mapping, printing, & public...",https://www.fbo.gov/index.php?s=opportunity&mo...,Government Publishing Office Customer Services...,Combined Synopsis/Solicitation (Modified),"Sep 04, 2019"
4,86614619Q00007,Student Housing Competition,"R -- Professional, administrative, and managem...",https://www.fbo.gov/index.php?s=opportunity&mo...,Department of Housing and Urban Development Fo...,Award / Woman Owned Small Business,"Sep 04, 2019"


In [29]:
link = df.detail_page[10]
link

'https://www.fbo.gov/index.php?s=opportunity&mode=form&id=b02beb005b9d973e3958e9465d5002ea&tab=core&_cview=1'

In [26]:
requests.compat.urljoin(url, link)

'https://www.fbo.gov/index.php?s=opportunity&mode=form&id=b02beb005b9d973e3958e9465d5002ea&tab=core&_cview=1'

## **3 개별 페이지 내용 살펴보기**
https://github.com/SmugZombie/FBO_Parser/blob/master/fbo_parse.py

In [30]:
#html 
#body.yui-skin-sam.html4.ff.ff2 
#div.sys-vendor div.sec-opportunity 
#div#bodywrap 
#div#c 
#div#cc 
#div.mc 
#table.layout 
#tbody tr td.sidebar div.sb

page_url = df.detail_page[9]
html = html_download(page_url)
dom  = BeautifulSoup(html.text, "html.parser") # 오류시 
type(html.text), type(dom), page_url

(str,
 bs4.BeautifulSoup,
 'https://www.fbo.gov/index.php?s=opportunity&mode=form&id=83e62421f423928d0f794c021304ac11&tab=core&_cview=1')

In [31]:
# 첨부파일 style class : subformclass_document_package
attached = dom.find('div', {"class":"subformclass_document_package"}) 

result   = []
if attached:
    add_files = attached.find_all("a", {"class":"file"})
    if len(add_files) > 0:
        result += [requests.compat.urljoin(page_url, _['href'])  for _ in add_files]
    add_links = attached.find_all("a", {"class":"link"})
    if len(add_links) > 0:
        result += [_['href']  for _ in add_links] # 외부 link 수집
    print(result)
else:
    print("link does not exist!")
# dom.find('div', {"class":"subformclass_document_package"}).find_all("a", {"class":"link"})

['https://www.fbo.gov/utils/view?id=5db1f6f772a6c197c251ab32aafd9f87']


In [32]:
# 첨부파일 요약 및 압축파일 찾기 : 하지만 2번 거쳐야 되고 포맷도 복잡해서 제외하기
# tags = dom.find('div', {"id":"bodywrap"}).find("div",{"id":"cc"}).\
#     find("table",{"class":"layout"}).find('td',{"class":"sidebar"}).find_all('div',{"class":"sb"})[0]
# for _ in tags.find_all('a'):
#     print(_.text, "||", _['href'], "\n")

# 수집할 대상의 특정
tags = dom.find('div', {"id":"bodywrap"}).find("div",{"id":"cc"}).\
    find("table",{"class":"layout"}).find('td',{"class":"sidebar"}).find_all('div',{"class":"sb"})[-1]

import unidecode
# ! pip install Unidecode
regex_token  = re.compile("so_formfield_dnf_class_values_procurement_notice__.*?")
# div id="so_formfield_dnf_class_values_procurement_notice__procurement_type_"
# div id="so_formfield_dnf_class_values_procurement_notice__posted_date_"
# div id="so_formfield_dnf_class_values_procurement_notice__response_deadline_"
regex_detail = re.compile(r"[^ (\n)(\t)]+", re.UNICODE)

texts = [_.text.strip()  for _ in tags.find_all("div", {"id":regex_token})]
texts = [" ".join(regex_detail.findall(str(_.text)))  for _ in tags.find_all("div", {"id":regex_token})]
texts = [unidecode.unidecode(_)  for _ in texts]  # /xa0 등 불필요한 유니코드 삭제
texts

['Notice Type: Solicitation',
 'Original Posted Date: September 3, 2019',
 'Posted Date: September 4, 2019',
 'Response Date: Sep 15, 2019 11:59 pm Central',
 'Original Response Date: Sep 15, 2019 11:59 pm Central',
 'Archiving Policy: Automatic, 15 days after response date',
 'Original Archive Date: September 30, 2019',
 'Archive Date: September 30, 2019',
 'Original Set Aside: Total Small Business',
 'Set Aside: Total Small Business',
 'Classification Code: 99 -- Miscellaneous',
 'NAICS Code: 311 -- Food Manufacturing/311812 -- Commercial Bakeries']